In [4]:
import scipy.io
import numpy as np
import theano
import theano.tensor as T
from theano.tensor.signal import pool
from theano.tensor.nnet import conv2d
import random

# import 5th layer data
force = scipy.io.loadmat('true.mat')
neurons = scipy.io.loadmat('hidstates5th_WB_recon_30neuron')
neuron5=neurons['xtr'].T
data_label=np.asarray(force['skt']).reshape(-1)
label_y=(data_label-np.mean(data_label))/np.std(data_label)

#import 4th layer data
neurons = scipy.io.loadmat('hidstates4th_WB_recon_1000neuron')
neuron4=neurons['xtr']
W_4to5=scipy.io.loadmat('W5.mat')['W5']

#import 3rd layer data
neurons = scipy.io.loadmat('hidstates3rd_WB_recon_373248neuron')
neuron3=neurons['xtr']
W_3to4=scipy.io.loadmat('W4.mat')['W4'] 

#import 2nd layer data
neurons = scipy.io.loadmat('hidstates2nd_WB_recon_77440neuron')
neuron2=neurons['pool']
W_2to3=scipy.io.loadmat('W3.mat')['W3'].reshape(288,40,9,9) #in the right shape

#import 1st layer data
neurons = scipy.io.loadmat('hidstates1st_WB_recon_225816neuron')
neuron1=neurons['pool']
W_1to2=scipy.io.loadmat('W2.mat')['W2'].reshape(40,24,9,9) #in the right shape

#import input data
neurons = scipy.io.loadmat('input_neuron')
neuron_input = neurons['WB']
W_0to1 = scipy.io.loadmat('W1.mat')['W1'].reshape(24,1,6,6)  #in the right shape



In [5]:
class RBM_NN(object):
    def __init__(self, input, n_in, n_out):
        self.W = theano.shared(value=np.zeros((n_in, n_out),dtype=theano.config.floatX),name='W',borrow=True)
        self.b = theano.shared(value=np.zeros((n_out,),dtype=theano.config.floatX),name='b',borrow=True)               
        self.y_pred = T.dot(input, self.W)+self.b      
        self.params = [self.W, self.b]
        
    def difference(self,y):
        return T.mean(abs(self.y_pred-T.reshape(y,[10,1])))
    
def load_data(data1, data2):
    def shared_dataset(data1,data2,borrow=True):
        shared_x = theano.shared(np.asarray(data1, dtype=theano.config.floatX),borrow=True)
        shared_y = theano.shared(np.asarray(data2, dtype=theano.config.floatX),borrow=True)
        return shared_x, T.cast(shared_y,'float64')
    
    train_set_x, train_set_y = shared_dataset(data1,data2)
    rval = [(train_set_x, train_set_y)]
    return rval

class HiddenLayer(object):
    def __init__(self,input,n_in,n_out,WW):
        self.input=input
        W=np.asarray(WW,dtype=theano.config.floatX)
        W=theano.shared(value=W, name='W',borrow=True)
        #W_values = np.ones((n_in,n_out), dtype=theano.config.floatX)
        #W = theano.shared(value=W_values, name='W', borrow=True)
                
        b_value = np.zeros((n_out,),dtype=theano.config.floatX)
        b = theano.shared(value=b_value, name='b',borrow=True)
        
        self.W=W
        self.b=b
        
        lin_output = T.dot(input,self.W) + self.b
        self.output = T.nnet.sigmoid(lin_output)
        
        self.params = [self.W, self.b]
        
class ConvPoolLayer(object):
    def __init__(self, input,filter_shape,image_shape, poolsize,WW):
        self.input = input
        self.W=theano.shared(np.asarray(WW,dtype=theano.config.floatX),
                             borrow=True)
        
        b_values = np.zeros((filter_shape[0],),dtype=theano.config.floatX)
        self.b = theano.shared(value=b_values, borrow=True)
        
        conv_out = conv2d(input=input,
                         filters=self.W,
                         filter_shape=filter_shape,
                         input_shape=image_shape)  
        
        pooled_out = pool.pool_2d(input = conv_out,
                                 ds=poolsize,
                                 ignore_border=True)
        
        
        self.output = T.nnet.sigmoid(pooled_out+self.b.dimshuffle('x',0,'x','x'))
        
        self.params = [self.W, self.b]
        
        self.input = input
        
        
class MLP(object):
    def __init__(self,input,n_in,n_hidden1, n_hidden2, n_hidden3, n_hidden4, n_hidden, n_out):
        
        self.inputLayer = ConvPoolLayer(input = input,
                                       image_shape = (10,1,200,200),
                                       filter_shape = (24,1,6,6),
                                       poolsize = (2,2),
                                       WW=W_0to1)
        
        self.hiddenLayer1 = ConvPoolLayer(input = self.inputLayer.output,
                                         image_shape = (10,24,97,97),
                                         filter_shape = (40,24,9,9),
                                         poolsize = (2,2),
                                         WW=W_1to2)    
        
        self.hiddenLayer2 = ConvPoolLayer(input = self.hiddenLayer1.output,
                                          image_shape = (10,40,44,44),
                                          filter_shape = (288,40,9,9),
                                          poolsize=(1,1),
                                          WW=W_2to3)
        
        self.layer3_input = self.hiddenLayer2.output.flatten(2)
        #layer3_input = layer0.output.flatten(2)
        
        
        self.hiddenLayer3 = HiddenLayer(
        input=self.layer3_input, n_in=n_hidden3, n_out=n_hidden4,WW=W_3to4)

        self.hiddenLayer = HiddenLayer(
        input=self.hiddenLayer3.output, n_in=n_hidden4, n_out=n_hidden,WW=W_4to5)
        
        self.layer6 = RBM_NN(
        input=self.hiddenLayer.output, n_in=n_hidden, n_out=n_out)
        
        
        self.error = self.layer6.difference
        
        self.WD = abs(self.inputLayer.W**2).sum() + abs(self.hiddenLayer1.W**2).sum() + abs(self.hiddenLayer2.W**2).sum() + \
                    abs(self.hiddenLayer3.W**2).sum() + abs(self.hiddenLayer.W**2).sum() + abs(self.layer6.W**2).sum()
        
        #self.error2 = abs(self.hiddenLayer.W).sum()+abs(self.layer6.W).sum()        
        #self.error3 = abs(self.hiddenLayer3.W).sum()+abs(self.hiddenLayer.W).sum()+abs(self.layer6.W).sum()
        
        self.params = self.inputLayer.params + self.hiddenLayer1.params + self.hiddenLayer2.params + \
                        self.hiddenLayer3.params + self.hiddenLayer.params + self.layer6.params
        
        self.input = input
        
def test_mlp(learning_rate=0.1, n_epochs=2000, batch_size=10, n_hidden1=97*97*24, n_hidden2=44*44*40, n_hidden3=36*36*288, 
             n_hidden4=1000, n_hidden=30, data1=neuron_input, data2=label_y):
    datasets = load_data(data1,data2)
    train_set_x, train_set_y = datasets[0]
    
    n_train_batches = train_set_x.get_value(borrow=True).shape[0] // batch_size
    
    ######################
    # BUILD ACTUAL MODEL #
    ######################
    print('... building the model')
    index = T.lscalar()
    x = T.matrix('x')
    y = T.vector('y')    
    
    layer0_input = x.reshape((batch_size,1, 200, 200))
    
    #layer2 = ConvPoolLayer(input = layer2_input,
    #                      image_shape = (batch_size,40,44,44),
    #                      filter_shape = (288,40,9,9))
    
    #layer3_input = layer0.output.flatten(2)
    
    #print('-----check')
    #return layer3_input
    
    classifier = MLP(input=layer0_input,
                    n_in=200*200*1,
                    n_hidden1 = n_hidden1,
                    n_hidden2 = n_hidden2,
                    n_hidden3 = n_hidden3,
                    n_hidden4 = n_hidden4,
                    n_hidden = n_hidden,
                    n_out=1)
    
    
    cost = (classifier.error(y)+0.0001 * classifier.WD)
    
    gparams = [T.grad(cost, param) for param in classifier.params]
    
    updates = [(param, param - learning_rate * gparam)
              for param, gparam in zip(classifier.params, gparams)
              ]
    
    train_model = theano.function(inputs=[index],
                                 outputs=cost,
                                 updates=updates,
                                 givens={x:train_set_x[index*batch_size:(index+1)*batch_size],
                                         y:train_set_y[index*batch_size:(index+1)*batch_size]}
                                 )

    ###############
    # TRAIN MODEL #
    ###############
    print('... training the model')
    
    epoch=0 
    while (epoch<n_epochs):
        epoch = epoch + 1
        #print epoch
        #if epoch != 1:
        #    W=test_mlp2()
            
        for minibatch_index in range(n_train_batches): 
            minibatch_avg_cost=train_model(minibatch_index)    
            # iteration number
            iter = (epoch - 1) * n_train_batches + minibatch_index
            #print iter
            if (iter + 1) % 5 == 0:
                # compute zero-one loss on validation set
                losses = [train_model(i) for i in range(n_train_batches)]
                this_loss = np.mean(losses)
                
                print "Epoch {0}, Minibatch {1}/{2}, Test Error= {3}".format(epoch,minibatch_index+1,n_train_batches,
                                                                       this_loss)
        #print classifier.params[0].get_value()
        #print classifier.params
    return classifier.params


In [ ]:
W=test_mlp()

... building the model
